In [ ]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict

import geopandas as gpd
import pandas as pd
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf

from retrospective.load.fia import load_fia_common_practice
from retrospective.load.geometry import load_supersections
from retrospective.common_practice.common_practice import subset_common_practice_data
from retrospective.utils import load_arb_fortypcds
from retrospective.utils import aa_code_to_ss_code

In [ ]:
import seaborn as sns

sns.set(font_scale=1.5)
sns.set_style("white")
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def compare_olaf_slag(our_slag, olaf, threshold=75):
    comparison = olaf.join(
        our_slag.set_index(["aa_code", "site_class"]),
        rsuffix="_ours",
        on=["aa_code", "site_class"],
    )
    comparison = comparison.dropna(
        subset=["slag_co2e_acre_ours", "slag_co2e_acre"]
    )  # TODO: has to do w those duplicate aa names

    rmse = (
        mean_squared_error(comparison["slag_co2e_acre_ours"], comparison["slag_co2e_acre"]) ** 0.5
    )
    print(f"SLAG RMSE (all AA): {rmse:.2f}")

    subset = comparison[comparison["cond_prop_group"] > threshold]
    subset_rmse = mean_squared_error(subset["slag_co2e_acre_ours"], subset["slag_co2e_acre"]) ** 0.5
    print(f"SLAG RMSE (AA with more thatn {threshold} cond_prop_sum): {subset_rmse:.2f}")
    return comparison


def get_bootstrap_bounds(
    data,
    agg_var="slag_co2e_acre",
    statistic="mean",
    n_obs=1000,
    weights=None,
    bootstrap_interval=[0.025, 0.5, 0.975],
):
    """calculate bootstraped confidence intervals for .agg([]) statistic.
    Also includes the ability to weight by a column.
    For the slag work, i've found that weighting by condprop_unadj helps get closer to ARB's numbers.
    """

    resample = [
        data.sample(len(data), weights=weights, replace=True)[agg_var].agg([statistic])[statistic]
        for i in range(n_obs)
    ]
    return pd.Series(resample).quantile(bootstrap_interval)

## Loading up the data


In [ ]:
conus = [
    "AL",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]
df = load_fia_common_practice(conus)

In [ ]:
supersections = load_supersections("https://storage.googleapis.com/carbonplan-data")

In [ ]:
df = gpd.sjoin(df, supersections[["SSection", "geometry"]], how="inner", op="within")
df = df.rename(columns={"index_right": "supersection_id"})

### Some ancillary files


In [ ]:
arb_lut = pd.read_csv("/home/jovyan/lost+found/2015_aa_lut.csv")
# this whole analysis excludes alaska -- we drop those assessment areas here so we dont have to wrap this bit in a try/except.
arb_lut = arb_lut[arb_lut["ss_code"] != 75]
olaf = pd.read_csv("/home/jovyan/lost+found/cleaned_olaf.csv")
arb_fortyps = load_arb_fortypcds()

## Objective: Getting the right subset of data

We've gathered that ARB CP values are based on all FIA plots measured within three windows of time:
[2002, 2012], [2006, 2012], or [2008, 2012]. Unfortunately, we do not know the time window on a per
supersection basis. So the first step is to try and infer the time window used for calculating
Common Practice.

We accomplish this by using one of the intermediate output from
[Olaf Kuegler](https://ww2.arb.ca.gov/our-work/programs/compliance-offset-program/compliance-offset-protocols/us-forest-projects/2015/common-practice-data),
the FIA statistician who ran the queries/analysis the yielded the official ARB CP values. As one of
his intermediary analysis steps, he produces `groupings.accdb`, which contains a per assessment sum
of the number of proportional conditions (`CONDPROP_UNADJ` in FIAdb) that feed into the later stages
of the analysis. This is helpful because we can subset the FIA data in different ways, sum up
`CONDPROP_UNADJ`, and compare it against Olaf's number. If we're doing the subsetting correctly, we
should get fairly comporable values.

Nothing fancy here -- just brute force iteration.

We'll store this output in the repository so that we can, if we choose, use the same temporal
constraint when exploring CP ourselves.


### A short aside

Part of why this whole thing took me so long to write is I completely re-worked how we subset the
underlying FIA data. Still not sure those changes were the best thing in the world, but at least
things are a little less brittle. The really _good_ innovation was removing the explicit concept of
assessment areas from the subsetting. Eventually, we're goign to want to aggregate by hand-rolled
fortypcds, so having everything depending on `aa_code` and `ss_code` just wasn't going to work in
the long run.

The new approach inovlves explicitly specifying criteria and passing those criteria and the
dataframe to a subsetting function. This is going to be pretty nice in the world where we're looking
up a handful of projects. But it turns out that it's super slow for doing the bulk iteration we're
going to do here. So rather than think carefully about optimizing the queries I just do one join
between the supersection dataframe and the condition-slag dataframe. This feels fine because
demonstrating that we can honest to god recreate the ARB's CP values, per supersection, is a top
priority.

We start by building these "criteria" dicts -- and then we iterate through those dicts sequentially.
Each dict contains four criterion:

- supersection_id: this limits the spatial extent of the query.
- temporal_filter: min and max MEASYEAR to consider -- we need to figure out which range gets us
  closest to ARB.
- fortypcds: this gets us to assessment_area/forest community.


In [ ]:
varying_year_criteria = []

for row in arb_lut.itertuples():

    supersection_id = row.ss_code
    site_class = row.site_class
    fortyps = arb_fortyps[row.aa_code]

    for min_year in [2002, 2006, 2008]:
        varying_year_criteria.append(
            {
                "supersection_id": supersection_id,
                "temporal_filter": {"min_year": min_year, "max_year": 2012},
                "site_class": site_class,
                "fortyps": fortyps,
                "aa_code": row.aa_code,
            }
        )

In [ ]:
conds_per_aa = defaultdict(dict)
error_supersections = []
for criteria in varying_year_criteria:
    try:
        subset = subset_common_practice_data(
            df,
            criteria["supersection_id"],
            criteria["temporal_filter"],
            criteria["fortyps"],
            criteria["site_class"],
        )
        cond_prop_sum = subset.CONDPROP_UNADJ.sum()
        olaf_val = olaf[
            (olaf["aa_code"] == criteria["aa_code"])
            & (olaf["site_class"] == criteria["site_class"])
        ]["cond_prop_group"].item()
        delta_olaf = cond_prop_sum - olaf_val
        conds_per_aa[(criteria["aa_code"], criteria["site_class"])][
            criteria["temporal_filter"]["min_year"]
        ] = abs(delta_olaf)
    except:
        # we have some more work to do with ss_codes 36 and 40 -- there is a whole mess of typos/spelling errors that crop up over and over...forthcoming!
        error_supersections.append(criteria["supersection_id"])

aa_best_yr = {
    k: min(v, key=v.get) for k, v in conds_per_aa.items()
}  # select key (year) where `delta_olaf` is smallest -- this means we can no go from aa : min_year

In [ ]:
display(
    f"Supersections {[x for x in set(error_supersections)]} did not work. If this message displays anything other than 36 and 40, something has gone wrong"
)

In [ ]:
optimal_criteria = []
error_supersections = []

for row in arb_lut.itertuples():
    try:
        supersection_id = row.ss_code
        site_class = row.site_class
        fortyps = arb_fortyps[row.aa_code]
        min_year = aa_best_yr[(row.aa_code, row.site_class)]
        optimal_criteria.append(
            {
                "supersection_id": supersection_id,
                "temporal_filter": {"min_year": min_year, "max_year": 2012},
                "site_class": site_class,
                "fortyps": fortyps,
                "aa_code": row.aa_code,
            }
        )
    except:
        error_supersections.append(row.ss_code)
display(
    f"Supersections {[x for x in set(error_supersections)]} did not work. If this message displays anything other than 36 and 40, something has gone wrong"
)

## Some more analysis!

Alright so we found the right criteria for subsetting the data -- let's subset! First, we start with
just naive averaging. It turns out this works pretty well. Note that I am also excluding a handful
of conds where CONDPROP_UNDADJ is really tiny. I've found that these examples just cause trouble.


In [ ]:
unweighted_records = []
for criteria in optimal_criteria:
    mean_slag = subset_common_practice_data(
        df.loc[df["CONDPROP_UNADJ"] > 0.005],
        criteria["supersection_id"],
        criteria["temporal_filter"],
        criteria["fortyps"],
        criteria["site_class"],
    ).slag_co2e_acre.mean()
    unweighted_records.append((criteria["aa_code"], criteria["site_class"], mean_slag))

In [ ]:
unweighted_slag = pd.DataFrame(
    unweighted_records, columns=["aa_code", "site_class", "slag_co2e_acre"]
)

In [ ]:
region_eight_problem_supersections = [
    87,
    71,
]  # worst offenders are 87, 71. Something goofy going on w parts of 46 & 28
xo_aa_codes = [
    k for k, v in aa_code_to_ss_code().items() if v in region_eight_problem_supersections
]

excluded_aa = unweighted_slag.loc[unweighted_slag["aa_code"].isin(xo_aa_codes)]
excluded = compare_olaf_slag(excluded_aa, olaf, threshold=30)

g = sns.lmplot(
    x="slag_co2e_acre_ours",
    y="slag_co2e_acre",
    data=excluded,
    hue="ss_code",
    fit_reg=False,
    ci=False,
    scatter_kws={"s": 100, "alpha": 0.55},
)
axes = g.axes.flatten()
[ax.set_xlim(0, 300) for ax in axes]
[ax.set_ylim(0, 300) for ax in axes]
axes[0].set_yticks(np.arange(0, 301, 100))
[ax.plot((0, 300), (0, 300), lw=3, ls="--", c="r") for ax in axes]

plt.xlabel("Recalculated Common Practice\n(t CO2e acre$^{-1}$)")
plt.ylabel("ARB Reported Common Practice\n(t CO2e acre$^{-1}$)")

I still can’t explain precisely what is going on, but I can convincingly say that the majority of
problems occur in USFS Region 8 [You might recall, Region 8 gave us a bunch of trouble for the
mortality work too.] I have two leading theories.

- We know that Region 8 has strange sampling patterns — they run out of money and don’t measure
  their trees. So it could be the case that when ARB made their numbers, they used a different
  subset of data than we’re using. I’m open to this interpretation but after poking around the data
  we’re using and the data posted on ARB’s website, I don’t see any crazy differences.
- “Access Denied” — Region 8 has lots and lots of plots that don’t get measured because of (i) the
  land owner won’t let them (see Texas; cross ref. the Myth of the Rugged Individual) or (ii)
  mash/wetland plots that are too dangerous to measure (see Crocodylus acutus). If the ARB’s common
  practice calculations attempt to account for missingness — and the missing plots look much
  different from sampled plots in terms of carbon stocks — this could drive our disagreements.
  Thankfully, we’re not trying to land on the moon and we just need to have a rough idea of what is
  going on. So we don’t actually have to discern between the two hypotheses above. Instead, we can
  just see what happens when we exclude i) coastal supersections and ii) a big Texas supersection
  where disagreeement is rife (and we know “refusals” are high). And lo (see attached), if we
  withhold 4 problematic supersections from our analysis, our performance in predicting common
  practice increases markedly.

N.B. SS 71 and 87 have southern yellow pine (slash/lob &etc) fortyps -- which means they could
include some of the productive-ish plantations in the region.


In [ ]:
subset_unweighted = unweighted_slag.loc[~unweighted_slag["aa_code"].isin(xo_aa_codes)]
subset = compare_olaf_slag(subset_unweighted, olaf)


def compare_cp_plot(data):
    g = sns.lmplot(
        x="slag_co2e_acre_ours",
        y="slag_co2e_acre",
        data=data,
        fit_reg=False,
        ci=False,
        scatter_kws={"s": 100, "color": ".3", "alpha": 0.55},
    )
    axes = g.axes.flatten()
    [ax.set_xlim(0, 300) for ax in axes]
    [ax.set_ylim(0, 300) for ax in axes]
    axes[0].set_yticks(np.arange(0, 301, 100))
    [ax.plot((0, 300), (0, 300), lw=3, ls="--", c="r") for ax in axes]
    rmse = mean_squared_error(data["slag_co2e_acre_ours"], data["slag_co2e_acre"]) ** 0.5

    axes[0].an
    plt.xlabel("Recalculated Common Practice\n(t CO2e acre$^{-1}$)")
    plt.ylabel("ARB Reported Common Practice\n(t CO2e acre$^{-1}$)")

## And all together


In [ ]:
res = compare_olaf_slag(unweighted_slag, olaf)

g = sns.lmplot(
    x="slag_co2e_acre_ours",
    y="slag_co2e_acre",
    data=res,
    fit_reg=False,
    ci=False,
    scatter_kws={"s": 100, "color": ".3", "alpha": 0.55},
)
axes = g.axes.flatten()
[ax.set_xlim(0, 300) for ax in axes]
[ax.set_ylim(0, 300) for ax in axes]
axes[0].set_yticks(np.arange(0, 301, 100))
[ax.plot((0, 300), (0, 300), lw=3, ls="--", c="r") for ax in axes]

plt.xlabel("Recalculated Common Practice\n(t CO2e acre$^{-1}$)")
plt.ylabel("ARB Reported Common Practice\n(t CO2e acre$^{-1}$)")

## Performance from lens of a linear model


In [ ]:
smf.ols("slag_co2e_acre~slag_co2e_acre_ours", data=res).fit().params

Small intercept and slope is pretty near one. Hurray! RMSE lf 10.11 (or close), which is reduced if
we chop off assessment areas with a low count.


In [ ]:
%%time
# with n_obs = 1000, this takes 13 minutes but is embarassingly dask-able if you're impatient. I just got get more coffee


weighted_records = {} # this is a little different because i want to keep around the bounds
for criteria in optimal_criteria:
    try:
        subset = subset_common_practice_data(df.loc[df['CONDPROP_UNADJ'] > 0.005], 
                                    criteria['supersection_id'], 
                                    criteria['temporal_filter'], 
                                    criteria['fortyps'],
                                    criteria['site_class'])

        res = get_bootstrap_bounds(subset, n_obs=250, weights='CONDPROP_UNADJ')
        weighted_records[(criteria['aa_code'], criteria['site_class'])] = res
    except:
        # caused by the ghost superseciton, Ouachita Mixed Forest
        print(criteria['aa_code'])

#weighted_slag = pd.DataFrame(weighted_records, columns=['aa_code', 'site_class', 'slag_co2e_acre'])

In [ ]:
weighted_bounds = pd.DataFrame(weighted_records).T
weighted_slag = (
    weighted_bounds[0.5]
    .reset_index()
    .rename(
        columns={
            "level_0": "aa_code",
            "level_1": "site_class",
            0.5: "slag_co2e_acre_ours",
        }
    )
)

In [ ]:
weighted_slag = weighted_slag.loc[~weighted_slag["aa_code"].isin(xo_aa_codes)]

In [ ]:
w_res = compare_olaf_slag(weighted_slag, olaf, threshold=30)


def compare_cp_plot(data):
    g = sns.lmplot(
        x="slag_co2e_acre_ours",
        y="slag_co2e_acre",
        data=data,
        fit_reg=False,
        ci=False,
        scatter_kws={"s": 100, "color": ".3", "alpha": 0.55},
    )
    axes = g.axes.flatten()
    [ax.set_xlim(0, 300) for ax in axes]
    [ax.set_ylim(0, 300) for ax in axes]
    axes[0].set_yticks(np.arange(0, 301, 100))
    [ax.plot((0, 300), (0, 300), lw=3, ls="--", c="r") for ax in axes]
    params = smf.ols("slag_co2e_acre~slag_co2e_acre_ours", data=data).fit().params
    rmse = mean_squared_error(data["slag_co2e_acre_ours"], data["slag_co2e_acre"]) ** 0.5

    axes[0].annotate(
        f"Intercept: {params['Intercept']:.2f}\nSlope: {params['slag_co2e_acre_ours']:.2f}\nRMSE: {rmse:.2f}",
        xy=(0.60, 0.25),
        xycoords="figure fraction",
    )

    plt.xlabel("Recalculated Common Practice\n(tCO2e acre$^{-1}$)")
    plt.ylabel("ARB Reported Common Practice\n(tCO2e acre$^{-1}$)")


compare_cp_plot(w_res)

Sampling really smoothes things out. It might not seem like much, but RMSE drops, the intercept gets
smaller, slope gets closer to one (in fact its > 1 now, driven by those handful of points where we
underestimate common practice).

This is great -- it builds confdience that we're capable of recreating CP. I should actually clean
this above figure up with some annotations etc.


In [ ]:
weighted_bounds = (
    pd.DataFrame(weighted_records)
    .T.reset_index()
    .rename(columns={"level_0": "aa_code", "level_1": "site_class"})
)

In [ ]:
full = olaf.join(
    weighted_bounds.set_index(["aa_code", "site_class"]),
    on=["aa_code", "site_class"],
)

In [ ]:
def fraction_within_ci(data):
    inside = data[(data["slag_co2e_acre"] > data[0.025]) & (data["slag_co2e_acre"] < data[0.975])]

    outside = data[
        (data["slag_co2e_acre"] <= data[0.025]) | (data["slag_co2e_acre"] >= data[0.975])
    ]
    display(
        f"{len(inside)/len(data)*100:.2f} percent of ARB CP values fall within our bootstrapped 95th confidence intervals of mean CP"
    )
    return (inside, outside)


inside, outside = fraction_within_ci(full.loc[~full.ss_code.isin([71, 87, 63])])

In [ ]:
def fraction_within_ci(data):
    inside = data[(data["slag_co2e_acre"] > data[0.025]) & (data["slag_co2e_acre"] < data[0.975])]

    outside = data[
        (data["slag_co2e_acre"] <= data[0.025]) | (data["slag_co2e_acre"] >= data[0.975])
    ]
    return len(inside) / len(data)


full.groupby("ss_code").apply(fraction_within_ci).sort_values().head(25)

## notes on assessment areas falling outside CI

Nothign stands out as remarkable for what makes these 70 SS difficult to get...

- not something systematic about sample size.
- nor is it something systematic about standard deviation/mean...

however, on balance, they seem to occur in USFS Region 8/Southern Section of FIA.


## What if we don't do the variable year thing?

What if we didnt bother to figure out the right time interval? We re-create our criteria filters,
but this time we fix `min_year = 2002`


In [ ]:
fixed_min_year = []
for row in arb_lut.itertuples():
    try:
        supersection_id = row.ss_code
        site_class = row.site_class
        fortyps = arb_fortyps[row.aa_code]
        min_year = aa_best_yr[(row.aa_code, row.site_class)]
        fixed_min_year.append(
            {
                "supersection_id": supersection_id,
                "temporal_filter": {"min_year": 2002, "max_year": 2012},
                "site_class": site_class,
                "fortyps": fortyps,
                "aa_code": row.aa_code,
            }
        )
    except:
        pass  # yolo this is just quick example

In [ ]:
fixed_year_records = []
for criteria in fixed_min_year:
    mean_slag = subset_common_practice_data(
        df.loc[df["CONDPROP_UNADJ"] > 0.005],
        criteria["supersection_id"],
        criteria["temporal_filter"],
        criteria["fortyps"],
        criteria["site_class"],
    ).slag_co2e_acre.mean()
    fixed_year_records.append((criteria["aa_code"], criteria["site_class"], mean_slag))

In [ ]:
fixed_slag = weighted_slag = pd.DataFrame(
    fixed_year_records, columns=["aa_code", "site_class", "slag_co2e_acre"]
)

f_res = compare_olaf_slag(fixed_slag, olaf)

g = sns.lmplot(
    x="slag_co2e_acre_ours",
    y="slag_co2e_acre",
    data=f_res,
    fit_reg=False,
    ci=False,
    scatter_kws={"s": 100, "color": ".3", "alpha": 0.55},
)
axes = g.axes.flatten()
[ax.set_xlim(0, 300) for ax in axes]
[ax.set_ylim(0, 300) for ax in axes]
axes[0].set_yticks(np.arange(0, 301, 100))
[ax.plot((0, 300), (0, 300), lw=3, ls="--", c="r") for ax in axes]

plt.xlabel("Recalculated Common Practice\n(t CO2e acre$^{-1}$)")
plt.ylabel("ARB Reported Common Practice\n(t CO2e acre$^{-1}$)")

In [ ]:
smf.ols("slag_co2e_acre~slag_co2e_acre_ours", data=f_res).fit().params

So yeah it turns out this isnt that big of a deal in aggreate. Intercept changes the most, but slope
is quite similar. In effect, this means that all the extra work in this notebook -- and the efforts
to make it so we can have variable aggregation through time are largely unneeded. But we didn't know
that beforehand!


ARB says in the future
[they'll include any assessment area with more than 30 FIA plots](https://ww2.arb.ca.gov/sites/default/files/classic//cc/capandtrade/offsets/guidance.document.amended.2015.pdf)
(!):

> Addition of new assessment areas will follow ARB protocols and will require a minimum of 2 years
> of data on a minimum of 30 FIA plots.


## Left over from debug TX

df[(df['ECOSUBCD'].str.strip().str[:4].isin(['255D', '255B', '255C'])) & (df['BALIVE'] >
0)].groupby('FORTYPCD').BALIVE.agg(['mean', 'count']) from retrospective.utils import
get_state_boundaries

bounds = get_state_boundaries(['tx', 'la', 'al', 'fl'])

tx_and_south_ss_codes = gpd.sjoin(bounds, supersections)['index_right'].unique().tolist()
display(tx_and_south_ss_codes)
